In [ ]:
# default_exp derivative

# default_cls_lvl 3

In [ ]:
#export
import xarray as xr
import numpy as np
from tqdm import tqdm

# Derivatives

> calculates derivatives from storm object

In [ ]:
#export
def harmonicRegression(self, data, N):
    """Returns Fourier coefficients"""
    result = np.zeros((N, 2))
    T = len(data)
    t = np.arange(T)
    for n in range(N):
        result[n,0] = 2/T*(data * np.cos(2*np.pi*n*t/T)).sum()
        result[n,1] = 2/T*(data * np.sin(2*np.pi*n*t/T)).sum()
    return result

def derivative(self, variable, N = 20, dim = "", order = 1):
    """
    Calculates the first and second derivative in dimension
    of choice, applying a harmonic regression.
    See Benestad et al. (2006)
    """
    dp = np.zeros((self.ds[variable].data.shape))

    if dim == self._time_name:
        print("Calculating d/dt for all lon, lat points ...")
        for i in tqdm(range(len(self.ds[self._latitude_name]))):
            for ii in range(len(self.ds[self._longitude_name])):
                dp[:, i, ii] =  self.calc_dX(self.ds[variable].data[:, i, ii], N, order)
    elif dim == self._latitude_name:
        print("Calculating d/dlat for all time, lon points ...")
        for i in tqdm(range(len(self.ds[self._time_name]))):
            for ii in range(len(self.ds[self._longitude_name])):
                dp[i, :, ii] =  self.calc_dX(self.ds[variable].data[i, :, ii], N, order)
    elif dim == self._longitude_name:
        print("Calculating d/dlon for all time, lat points ...")
        for i in tqdm(range(len(self.ds[self._time_name]))):
            for ii in range(len(self.ds[self._latitude_name])):
                dp[i, ii, :] =  self.calc_dX(self.ds[variable].data[i, ii, :], N, order)
    else:
        errmsg = "{} is not found.".format(dim)

    if order == 1:
        self.ds['dd{}{}'.format(dim, variable)] = ([self._time_name, self._latitude_name, self._longitude_name], dp)
    if order == 2:
        self.ds['d2d{}{}'.format(dim, variable)] = ([self._time_name, self._latitude_name, self._longitude_name], dp)

def calc_dX(self, data, N, order):
    """
    Calculation of the derivative using harmonic regression (Fourier fit)
    """
    if np.isnan(data).all() == True:
        return

    mask = np.isfinite(data)
    T = len(data[mask])
    t = np.arange(T)

    data = data - data.mean()

    if N > T:
        N = T

    anbn = self.harmonicRegression(data[mask], N)

    Wi = 2 * np.pi / T

    dp = 0

    if order == 1:
        for i in range(0, N):
            dp = dp + (Wi * (-anbn[i,0]*np.sin(Wi*i*t)+anbn[i,1]*np.cos(Wi*i*t)))
    elif order == 2:
        for i in range(0, N):
            dp = dp + (Wi**2 * (-anbn[i,0]*np.cos(Wi*i*t) - anbn[i,1]*np.sin(Wi*i*t)))
    else:
        print("Your selection of order {} is not possible. Only 1th and 2nd derivative possible.")

    tmp = data.copy()
    tmp[mask] = dp

    return tmp

In [ ]:
#hide
from nbdev import *

In [ ]:
show_doc(calc_dX)
show_doc(harmonicRegression)
show_doc(derivative)

<h4 id="calc_dX" class="doc_header"><code>calc_dX</code><a href="__main__.py#L43" class="source_link" style="float:right">[source]</a></h4>

> <code>calc_dX</code>(**`data`**, **`N`**, **`order`**)

Calculation of the derivative using harmonic regression (Fourier fit)

<h4 id="harmonicRegression" class="doc_header"><code>harmonicRegression</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>harmonicRegression</code>(**`data`**, **`N`**)

Returns Fourier coefficients

<h4 id="derivative" class="doc_header"><code>derivative</code><a href="__main__.py#L12" class="source_link" style="float:right">[source]</a></h4>

> <code>derivative</code>(**`variable`**, **`N`**=*`20`*, **`dim`**=*`''`*, **`order`**=*`1`*)

Calculates the first and second derivative in dimension 
of choice, applying a harmonic regression.
See Benestad et al. (2006)        

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_storm_object.ipynb.
Converted 01_derivatives.ipynb.
Converted index.ipynb.
